In [122]:
import pandas as pd
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
dim_composer_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_composer.csv')
dim_rating_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_rating.csv')
dim_staff_category_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_staff_category.csv')
dim_staff_film_bridge_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_staff_film_bridge.csv')
dim_staff_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_staff.csv')
dim_type_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_type.csv')
dim_year_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/dim_year.csv')
fact_film_df = pd.read_csv(r'/Users/shuya/Desktop/DEP Final Project/fact_film.csv')

In [123]:
film_composer_df = pd.merge(fact_film_df, dim_composer_df, left_on='composer_key', right_on='composer_key', how='left')

film_rating_df = pd.merge(film_composer_df, dim_rating_df, left_on='rating_key', right_on='rating_key', how='left')

film_type_df = pd.merge(film_rating_df, dim_type_df, left_on='type_key', right_on='type_key', how='left')

film_year_df = pd.merge(film_type_df, dim_year_df, left_on='start_year_key', right_on='year_key', how='left')

film_staff_bridge_df = pd.merge(fact_film_df, dim_staff_film_bridge_df, left_on='film_id', right_on='film_id', how='left')

film_staff_df = pd.merge(film_staff_bridge_df, dim_staff_df, left_on='staff_key', right_on='staff_key', how='left')

film_staff_category_df = pd.merge(film_staff_df, dim_staff_category_df, left_on='staff_category_key', right_on='staff_category_key', how='left')

In [124]:
film_staff_category_df.head()

,film_id,title,type_key,start_year_key,rating_key,composer_key,episode_key,isadult,run_time,genres,staff_key,staff_category_key,characters,first_name,last_name,birth_year,profession,known_for,category
0,tt0000001,Carmencita,1,7.0,1.0,1.0,NaN,0,1,Documentary,nm0005690,2.0,NaN,NaN,NaN,NaN,NaN,NaN,director
1,tt0000002,Le clown et ses chiens,1,5.0,2.0,NaN,NaN,0,5,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0000003,Pauvre Pierrot,1,5.0,3.0,NaN,NaN,0,4,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000004,Un bon bock,1,5.0,4.0,NaN,NaN,0,12,Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0000005,Blacksmith Scene,1,6.0,5.0,2.0,NaN,0,1,Comedy,nm0005690,2.0,NaN,NaN,NaN,NaN,NaN,NaN,director


In [174]:
#Create a new DataFrame to store the feature matrix
feature_matrix = pd.DataFrame(index=film_staff_category_df['film_id'].unique())

In [175]:
# Add genre features
genre_dummies = pd.get_dummies(film_staff_category_df['genres'])
feature_matrix = feature_matrix.join(genre_dummies).fillna(0)

#Add actor features
actors = film_staff_category_df[film_staff_category_df['profession'].isin(['actor', 'actress'])]
actor_dummies = pd.get_dummies(actors['first_name'] + ' ' + actors['last_name'])
feature_matrix = feature_matrix.join(actor_dummies, rsuffix='_actor').fillna(0)

#Add composer features
composer = film_staff_category_df[film_staff_category_df['profession'] == 'composer']
composer_dummies = pd.get_dummies(composer['first_name'] + ' ' + composer['last_name'])
feature_matrix = feature_matrix.join(composer_dummies, rsuffix='_composer').fillna(0)

#Add writer features
writer = film_staff_category_df[film_staff_category_df['profession'] == 'writer']
writer_dummies = pd.get_dummies(writer['first_name'] + ' ' + writer['last_name'])
feature_matrix = feature_matrix.join(writer_dummies, rsuffix='_writer').fillna(0)

#Add year features
feature_matrix['year'] = film_year_df.set_index('film_id')['year']

#Add rating features
feature_matrix['rating'] = film_rating_df.set_index('film_id')['rating']

# Standardize year and rating
feature_matrix['year'] = (feature_matrix['year'] - feature_matrix['year'].min()) / (feature_matrix['year'].max() - feature_matrix['year'].min())
feature_matrix['rating'] = feature_matrix['rating'] / feature_matrix['rating'].max()


In [184]:
#Weight features
weights = {'genre': 0.2, 'actor': 0.2, 'composer': 0.2, 'writer':0.1,'year': 0.1, 'rating': 0.2}
for feature in genre_dummies:
    feature_matrix[feature] *= weights['genre']
for feature in actor_dummies:
    feature_matrix[feature] *= weights['actor']
for feature in composer_dummies:
    feature_matrix[feature] *= weights['composer']
for feature in writer_dummies:
    feature_matrix[feature] *= weights['writer']
feature_matrix['year'] *= weights['year']
feature_matrix['rating'] *= weights['rating']

In [185]:
feature_matrix.head()

,10,15,3,60,8,Action,Adventure,Animation,Biography,Comedy,...,Short,War,Lionel Barrymore,Edgar Poe,L. Baum,William Shakespeare,year,rating,total_score,similarity_score
tt0000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000204,0.025618,0.961714,0.701453
tt0000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.026067,0.967781,0.707107
tt0000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.029213,0.999242,0.707107
tt0000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.024719,0.954298,0.707107
tt0000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000102,0.027865,0.985205,0.704513


In [186]:
print(feature_matrix.isnull().sum())  # Displays the number of NaN values per column

10                     0
15                     0
3                      0
60                     0
8                      0
Action                 0
Adventure              0
Animation              0
Biography              0
Comedy                 0
Crime                  0
Documentary            0
Drama                  0
Family                 0
Fantasy                0
History                0
Horror                 0
Music                  0
Musical                0
Mystery                0
N                      0
News                   0
Romance                0
Sci-Fi                 0
Short                  0
War                    0
Lionel Barrymore       0
Edgar Poe              0
L. Baum                0
William Shakespeare    0
year                   0
rating                 0
total_score            0
similarity_score       0
dtype: int64


In [187]:
feature_matrix = feature_matrix.dropna(axis=1, how='all')  # Deletes columns that are all Nans
feature_matrix = feature_matrix.fillna(0)  # Replace the NaN value with 0

In [188]:
print(set(user_feature_vector.index) - set(feature_matrix.columns))  # Find columns that exist in the user_feature_vector that do not exist in feature_matrix
print(feature_matrix.describe())  # Check for outliers

set()
           10      15       3      60       8  Action  Adventure  Animation  \
count  2000.0  2000.0  2000.0  2000.0  2000.0  2000.0     2000.0     2000.0   
mean      0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
std       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
min       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
25%       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
50%       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
75%       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   
max       0.0     0.0     0.0     0.0     0.0     0.0        0.0        0.0   

       Biography  Comedy  ...   Short     War  Lionel Barrymore  Edgar Poe  \
count     2000.0  2000.0  ...  2000.0  2000.0            2000.0     2000.0   
mean         0.0     0.0  ...     0.0     0.0               0.0        0.0   
std          0.0     0.0  ...     0.0     0.0   

In [189]:
# user preference
user_preferences = {
    'genre_Action': 1, #assume the person likes Action movies
    'actress_Lauren': 1, #assume the person likes actress Lauren
    'composer': 0, #assume the person doesn't choose a composer 
    'writer_Ingmar': 0, # assume the perosn likes the write Ingmar
    'year_1924': 1, #assume the person like the movie produced in 1924
    'rating': 1 #assume the peroson like the higher rating movies
}

# Create a user feature vector
# Make sure that the column names align with feature_matrix
user_feature_vector = pd.Series(0, index=feature_matrix.columns)
for feature in user_feature_vector.index:
    if feature in user_preferences:
        user_feature_vector[feature] = user_preferences[feature]

# Clean the NaN value in feature_matrix
feature_matrix = feature_matrix.fillna(0)
\
# Calculate the cosine similarity
cosine_similarities = cosine_similarity(feature_matrix, [user_feature_vector.values])

# Adds the similarity score to the feature matrix
feature_matrix['similarity_score'] = cosine_similarities.flatten()

# Sort by similarity score and output recommendations
recommendations = feature_matrix.sort_values(by='similarity_score', ascending=False)
print(recommendations.head(5))

            10   15    3   60    8  Action  Adventure  Animation  Biography  \
tt0000618  0.0  0.0  0.0  0.0  0.0     0.0        0.0        0.0        0.0   
tt0001456  0.0  0.0  0.0  0.0  0.0     0.0        0.0        0.0        0.0   
tt0000782  0.0  0.0  0.0  0.0  0.0     0.0        0.0        0.0        0.0   
tt0001089  0.0  0.0  0.0  0.0  0.0     0.0        0.0        0.0        0.0   
tt0000417  0.0  0.0  0.0  0.0  0.0     0.0        0.0        0.0        0.0   

           Comedy  ...  Short  War  Lionel Barrymore  Edgar Poe  L. Baum  \
tt0000618     0.0  ...    0.0  0.0               0.0        0.0      0.0   
tt0001456     0.0  ...    0.0  0.0               0.0        0.0      0.0   
tt0000782     0.0  ...    0.0  0.0               0.0        0.0      0.0   
tt0001089     0.0  ...    0.0  0.0               0.0        0.0      0.0   
tt0000417     0.0  ...    0.0  0.0               0.0        0.0      0.0   

           William Shakespeare      year    rating  total_score  \
t